# Joining and Appending DataFrames in PySpark

In this notebook we will be reviewing the foundational concepts of joinings and appending dataframes as well as the necessary PySpark calls to accomplish these tasks. 

So let's start!

In [0]:
# import findspark
# findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("joins").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

## Generate play data

First some play data to help us grasp some concepts. Let's create a database that has two tables. 

**Key Terms**
 - **omnivore**: an animal which is able to consume both plants (like a herbivore) and meat (like a carnivore)
 - **herbivore**: any animal that eats only vegetation (i.e. that eats no meat)
 - **carnivore**: any animal that eats meat as the main part of its diet

In [0]:
valuesP = [('koala',1,'yes'),('caterpillar',2,'yes'),('deer',3,'yes'),('human',4,'yes')]
eats_plants = spark.createDataFrame(valuesP,['name','id','eats_plants'])

valuesM = [('shark',5,'yes'),('lion',6,'yes'),('tiger',7,'yes'),('human',4,'yes')]
eats_meat = spark.createDataFrame(valuesM,['name','id','eats_meat'])

# print("Plant eaters (herbivores)")
# print(eats_plants.show())
# print("Meat eaters (carnivores)")
# print(eats_meat.show())

## Appends

Appending "appends" two dataframes together that have the exact same variables. It is like stacking two or more blocks ON TOP of each other.

In [0]:
import pandas as pd

In [0]:
# So first replicate table and call it new_df
new_df = eats_plants
# Then append using the union function
# this naming convention can be tricky to grasp for SQL enthusiasts 
# Where union just mean join
df_concat = eats_plants.union(new_df)
# We will test to see if this worked by getting before and after row counts
print(("eats_plants df Counts:", eats_plants.count(), len(eats_plants.columns)))
print(("df_concat Counts:", df_concat.count(), len(df_concat.columns)))
print(eats_plants.show(5))
print(df_concat.show(5))

('eats_plants df Counts:', 4, 3)
('df_concat Counts:', 8, 3)
+-----------+---+-----------+
|       name| id|eats_plants|
+-----------+---+-----------+
|      koala|  1|        yes|
|caterpillar|  2|        yes|
|       deer|  3|        yes|
|      human|  4|        yes|
+-----------+---+-----------+

None
+-----------+---+-----------+
|       name| id|eats_plants|
+-----------+---+-----------+
|      koala|  1|        yes|
|caterpillar|  2|        yes|
|       deer|  3|        yes|
|      human|  4|        yes|
|      koala|  1|        yes|
+-----------+---+-----------+
only showing top 5 rows

None


## Inner Joins!

Inner joins get us ONLY the values that appear in BOTH tables we are joining.

In [0]:
inner_join = eats_plants.join(eats_meat, ["name","id"],"inner")
print("Inner Join Example")
print(inner_join.show())
# So this is the only name that appears in BOTH dataframes

Inner Join Example
+-----+---+-----------+---------+
| name| id|eats_plants|eats_meat|
+-----+---+-----------+---------+
|human|  4|        yes|      yes|
+-----+---+-----------+---------+

None


## Left Joins

Left joins get us the values that appear in the left table and nothing additional from the right table except for its columns. A quick quality check we could do would be to make sure that the human column has the value "yes" for both eats_plants and eats_meat columns.

In [0]:
left_join = eats_plants.join(eats_meat, ["name","id"], how='left') # Could also use 'left_outer'
print("Left Join Example")
print(left_join.show())


Left Join Example
+-----------+---+-----------+---------+
|       name| id|eats_plants|eats_meat|
+-----------+---+-----------+---------+
|      koala|  1|        yes|     null|
|caterpillar|  2|        yes|     null|
|       deer|  3|        yes|     null|
|      human|  4|        yes|      yes|
+-----------+---+-----------+---------+

None


## Conditional Joins

Conditional joins have some additional logic that was not encompassed in the underlying join. For example, if we wanted to get all the values that appear in the left, **except** for those values that appear in BOTH tables, we could do this. Notice how human is left out now.

In [0]:
conditional_join = eats_plants.join(eats_meat, ["name","id"], how='left').filter(eats_meat.name.isNull())
print("Conditional Left Join")
print(conditional_join.show())

Conditional Left Join
+-----------+---+-----------+---------+
|       name| id|eats_plants|eats_meat|
+-----------+---+-----------+---------+
|      koala|  1|        yes|     null|
|caterpillar|  2|        yes|     null|
|       deer|  3|        yes|     null|
+-----------+---+-----------+---------+

None


## Right Join

A right join gets us the values that appear in the right table but not in the left. It also brings it's columns over of course.

In [0]:
right_join = eats_plants.join(eats_meat,  ["name","id"],how='right') # Could also use 'right_outer'
print("Right Join")
print(right_join.show())

Right Join
+-----+---+-----------+---------+
| name| id|eats_plants|eats_meat|
+-----+---+-----------+---------+
|shark|  5|       null|      yes|
| lion|  6|       null|      yes|
|tiger|  7|       null|      yes|
|human|  4|        yes|      yes|
+-----+---+-----------+---------+

None


## Full Outer Joins

Full outer joins will get all values from both tables, but notice that if there is a column that is common in both tables (ie. id and name in this case) that the join will take the value of the left table (see human id is p4 and not m4).

In [0]:
full_outer_join = eats_plants.join(eats_meat, ["name","id"],how='full') # Could also use 'full_outer'
print("Full Outer Join")
print(full_outer_join.show())

Full Outer Join
+-----------+---+-----------+---------+
|       name| id|eats_plants|eats_meat|
+-----------+---+-----------+---------+
|caterpillar|  2|        yes|     null|
|       deer|  3|        yes|     null|
|      human|  4|        yes|      yes|
|      koala|  1|        yes|     null|
|       lion|  6|       null|      yes|
|      shark|  5|       null|      yes|
|      tiger|  7|       null|      yes|
+-----------+---+-----------+---------+

None


## With REAL data

Thinking about how to join our data in real life will not be as easy as the above. We need to consider multiple aspects as we join tables in real life and ALWAYS conduct sanity checks to make sure we did it correctly. Let's look at an example with real data.

#### First, let's read in the datasets we will be working with

Here is a neat function that will read in all the csv files from a directory (folder) in one shot and returns a separate dataframe for each dataset in the directory using the same naming convention. This is super useful if you have a large set of files and don't feel like writing a separate line for each dataset in the directory.

In [0]:
dbfs_path = "/FileStore/shared_uploads/purvajainpj123@gmail.com"

df_list = []
for filename in dbutils.fs.ls(dbfs_path):
  if filename.name.endswith(".csv"):
    df_name = filename.name.split(".")[0]
    df = spark.read.csv(filename.path, header=True, inferSchema=True)
    df_list.append(df_name)
    globals()[df_name] = df

# QA
print("Full list of dfs:")
print(df_list)


Full list of dfs:
['13_SMSSpamCollection', 'Hack_data-1', 'Hack_data-2', 'Hack_data', 'Musical_instruments_reviews', 'course_offerings', 'courses', 'customer_churn', 'grade_distributions', 'instructors', 'iris-1', 'iris', 'iris_dataset_new-1', 'iris_dataset_new', 'new_customers', 'nyc', 'rooms', 'schedules', 'sections', 'subject_memberships', 'subjects', 'teachings']


## About this database

There are several tables that were read in above. This database will let us get a chance to practice our own custom joins and learn how the relationships between a real database work. Sometimes we don't know how they are related and we need to figure it out! 

For this we will focus on the 4 datasets below and save the rest for the HW. Here is a look at some of the important variables we will be using to join our tables:

 - **course_offerings:** uuid, course_uuid, term_code, name
 - **instructors:** id, name
 - **sections:** uuid, course_offering_uuid,room_uuid, schedule_uuid
 - **teachings:** instructor_id, section_uuid
 
 **Source:** https://www.kaggle.com/Madgrades/uw-madison-courses
 
Let's pretend that I am a student interested in seeing what courses are available. I suppose I would start by look at the course offerings table.

In [0]:
# View the data
course_offerings.limit(4).toPandas()

,uuid,course_uuid,term_code,name
0,344b3ebe-da7e-314c-83ed-9425269695fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1092,Cooperative Education Prog
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1082,Cooperative Education Prog
2,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog
3,075da420-5f49-3dd0-93df-13e3c152e1b1,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1114,Cooperative Education Prog


This course offers table is great, but I also want to know who teaches each course because I want to check the reviews of the instructor before I take the course. Let's see if we can join this table with the instructors table that contains the name of the instructor.

In [0]:
instructors.show(4,False)

+-------+------------------+
|id     |name              |
+-------+------------------+
|761703 |JOHN ARCHAMBAULT  |
|3677061|STEPHANIE KANN    |
|788586 |KATHY PREM        |
|1600463|KRISTIN KLARKOWSKI|
+-------+------------------+
only showing top 4 rows



So this table only contains 2 columns (id and name) and doesn't have the uuid or course uuid to join on. So we will need to see how we can accomplish the join we need. It looks like from the tables we have, we would need to take the following steps to get the variables we need. 

 - **course_offerings (CO):** uuid, course_uuid, term_code, name
 - **instructors (I):** id, name
 - **sections (S):** uuid, course_offering_uuid,room_uuid, schedule_uuid
 - **teachings (T):** instructor_id, section_uuid
 
 I.id --> T.instructor_id
                \/
          T.section_uuid --> S.uuid
                              \/
                             S.course_offering_uuid --> CO.uuid

In [0]:
teachings.show(3)

+-------------+--------------------+
|instructor_id|        section_uuid|
+-------------+--------------------+
|       761703|45adf63c-48c9-365...|
|       761703|c6280e23-5e43-385...|
|       761703|9395dc21-15d1-3fa...|
+-------------+--------------------+
only showing top 3 rows



In [0]:
# Let's try to see all course offerings and who teaches it
# Notice here that the variable we want to join on is different in the two datasets. 
# PySpark makes it easy to account for that
step1 = teachings.join(instructors, teachings.instructor_id == instructors.id, how='left').select(['instructor_id','name','section_uuid'])
step1.limit(4).toPandas()

,instructor_id,name,section_uuid
0,761703,JOHN ARCHAMBAULT,45adf63c-48c9-3659-8561-07556d2d4ddf
1,761703,JOHN ARCHAMBAULT,c6280e23-5e43-3859-893e-540d94993529
2,761703,JOHN ARCHAMBAULT,9395dc21-15d1-3fab-8d1f-6f3fe6114c48
3,3677061,STEPHANIE KANN,b99e440b-39db-350a-81eb-b6eb1bd8b0bc


In [0]:
step2 = step1.join(sections, step1.section_uuid == sections.uuid, how='left').select(['name','course_offering_uuid'])
step2.limit(4).toPandas()

,name,course_offering_uuid
0,JAMES STEELE,dfac15fb-e446-339e-9403-38b270895b6c
1,TERESA CLARK,878d4f26-4e7e-3cec-b2e3-28fd56d6489c
2,JAMES STEELE,3fc6bfe1-7929-3f2e-af13-5185f1cf7383
3,STEPHANIE KANN,ea3b717c-d66b-30dc-8b37-964d9688295f


In [0]:
step3 = step2.withColumnRenamed('name', 'instructor').join(course_offerings, step2.course_offering_uuid == course_offerings.uuid, how='inner').select(['instructor','name','course_offering_uuid'])
step3.limit(4).toPandas()

,instructor,name,course_offering_uuid
0,KAREN MC SHANE-HELLENBRAND,Modern Dance I,96f56d84-8c29-3dd5-8cf0-48608af3cbd2
1,MICHAEL FRANKLIN LORENZ,University Band,e32df8d0-c2ee-3d6c-acab-1c8b54f8506a
2,BEVERLY TAYLOR,Choral Union,039866e7-d700-3146-82ab-57b7a92813a7
3,MARTIN PICKENS,Intro to Speech Composition,41e6b821-0d66-3ecc-925c-ffb16de7ab1d


## One final really cool way to join datasets: The Levenshtien distance!

Which basically counts the number of edits we would need to make too strings equal to eachother.

In [0]:
# Compute the levenshtein distance beween two strings
# pyspark.sql.functions.levenshtein(left, right)  

from pyspark.sql.functions import levenshtein

df0 = spark.createDataFrame([('Aple', 'Apple','Microsoft','IBM')], ['Input', 'Option1','Option2','Option3'])
print("Correct this company name: Aple")
df0.select(levenshtein('Input', 'Option1').alias('Apple')).show()
df0.select(levenshtein('Input', 'Option2').alias('Microsoft')).show()
df0.select(levenshtein('Input', 'Option3').alias('IBM')).show()

Correct this company name: Aple
+-----+
|Apple|
+-----+
|    1|
+-----+

+---------+
|Microsoft|
+---------+
|        9|
+---------+

+---+
|IBM|
+---+
|  4|
+---+

